TS1-DAM

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime as dt

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/knn_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/SH_A_Q_DAM_1-12.csv")
dat1 = pd.DataFrame(dat)
# dat1 = dat1.iloc[152:, :].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r = dat1[column_names].dropna().stack().reset_index()

Y_r["Price"] = Y_r.iloc[:, 2:3]
Q_10["Price"] = Q_10.iloc[:, 2:3]
Q_30["Price"] = Q_30.iloc[:, 2:3]
Q_50["Price"] = Q_50.iloc[:, 2:3]
Q_70["Price"] = Q_70.iloc[:, 2:3]
Q_90["Price"] = Q_90.iloc[:, 2:3]


def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    day_index = df['level_0'].unique()

    for day in day_index:
        current_df = df[df['level_0'] == day]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == day]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == day]
        
        # Find the maximum price for that day  
        max_price_index = current_Q_A['Price'].idxmax()
        # Establish all the remaining prices for that day that fall before the max_price (for finding min price)
        prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index)]
        # Find the minimum price for that day  
        min_price_index = current_Q_B['Price'].idxmin()
        # Establish all remaining prices for that day that fall after the min_price (for finding max price)
        prices_after_min = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None
        # Identifying the min price for the remaining prices before the max
        if len(prices_before_max) > 0:
            min_price_index1 = prices_before_max['Price'].idxmin()
        # Identifying the max price for the remaining prices after the min
        if len(prices_after_min) > 0:
            max_price_index1 = prices_after_min['Price'].idxmax()
            
        # Handling potential missing values
        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            # Choose the pair with the greater difference
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                chosen_max_price_index = max_price_index
                chosen_min_price_index = min_price_index1
            else:
                chosen_max_price_index = max_price_index1
                chosen_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            chosen_max_price_index = max_price_index
            chosen_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            chosen_max_price_index = max_price_index1
            chosen_min_price_index = min_price_index

        # Calculate expected profit using predictions to decide if a trade should occur
        if chosen_max_price_index in current_Q_A.index and chosen_min_price_index in current_Q_B.index:
            Exp_profit = ((current_Q_A.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[chosen_min_price_index, 'Price']) / eff_2)
            
            # Only proceed to calculate actual profit if expected profit meets the threshold
            if Exp_profit >= -10:
                # Calculate the actual profit based on the observed prices
                profit = ((current_df.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index, 'Price']) / eff_2)
                prices.append((chosen_min_price_index, current_df.loc[chosen_min_price_index, 'Price'], chosen_max_price_index, current_df.loc[chosen_max_price_index, 'Price'], profit))

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])


# Running the strategy with various quantile combinations
r_dam_50_50 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90 = run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_DAM = run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r, eff_1=1, eff_2=1)

# Printing total profits for each combination
print(np.round(sum(r_dam_50_50.iloc[:, 4:5].values), 2))
print(np.round(sum(r_dam_10_30.iloc[:, 4:5].values), 2))
print(np.round(sum(r_dam_30_50.iloc[:, 4:5].values), 2))
print(np.round(sum(r_dam_50_70.iloc[:, 4:5].values), 2))
print(np.round(sum(r_dam_70_90.iloc[:, 4:5].values), 2))
print(np.round(sum(r_dam_30_70.iloc[:, 4:5].values), 2))
print(np.round(sum(r_dam_10_90.iloc[:, 4:5].values), 2))
print(np.round(sum(PF_DAM.iloc[:, 4:5].values), 2))


TS1 BM

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/knn_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/lgbm_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/MH_A_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
# dat1=dat1.iloc[456:,:].reset_index(drop=True)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]


def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]

        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                chosen_max_price_index = max_price_index
                chosen_min_price_index = min_price_index1
            else:
                chosen_max_price_index = max_price_index1
                chosen_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            chosen_max_price_index = max_price_index
            chosen_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            chosen_max_price_index = max_price_index1
            chosen_min_price_index = min_price_index

        # Calculate expected profit using predictions to decide if a trade should occur
        if chosen_max_price_index in current_Q_A.index and chosen_min_price_index in current_Q_B.index:
            Exp_profit = ((current_Q_A.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[chosen_min_price_index, 'Price']) / eff_2)
            
            # Only proceed to calculate actual profit if expected profit meets the threshold
            if Exp_profit >= 0:
                # Calculate the actual profit based on the observed prices
                profit = ((current_df.loc[chosen_max_price_index, 'Price']) * eff_1) - ((current_df.loc[chosen_min_price_index, 'Price']) / eff_2)
                prices.append((chosen_min_price_index, current_df.loc[chosen_min_price_index, 'Price'], chosen_max_price_index, current_df.loc[chosen_max_price_index, 'Price'], profit))

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])


r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=1, eff_2=1)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

TS2 DAM

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import datetime as dt

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/knn_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/SH_A_Q_DAM_1-12.csv")
dat1 = pd.DataFrame(dat)
# dat1 = dat1.iloc[152:, :].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90 = dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r = dat1[column_names].dropna().stack().reset_index()

Y_r["Price"] = Y_r.iloc[:, 2:3]
Q_10["Price"] = Q_10.iloc[:, 2:3]
Q_30["Price"] = Q_30.iloc[:, 2:3]
Q_50["Price"] = Q_50.iloc[:, 2:3]
Q_70["Price"] = Q_70.iloc[:, 2:3]
Q_90["Price"] = Q_90.iloc[:, 2:3]


def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    day_index = df['level_0'].unique()

    for day in day_index:
        current_df = df[df['level_0'] == day]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == day]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == day]
        
        # Find the maximum price for that day          
        max_price_index = current_Q_A['Price'].idxmax()
        prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        prices_after_min = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None
        
        if len(prices_before_max) > 0:
            min_price_index1 = prices_before_max['Price'].idxmin()
        if len(prices_after_min) > 0:
            max_price_index1 = prices_after_min['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                T1_max_price_index = max_price_index
                T1_min_price_index = min_price_index1
            else:
                T1_max_price_index = max_price_index1
                T1_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            T1_max_price_index = max_price_index
            T1_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            T1_max_price_index = max_price_index1
            T1_min_price_index = min_price_index

        # Calculate expected profit using predictions for T1
        if T1_max_price_index in current_Q_A.index and T1_min_price_index in current_Q_B.index:
            Exp_profit_T1 = ((current_Q_A.loc[T1_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[T1_min_price_index, 'Price']) / eff_2)
            if Exp_profit_T1 >= -10:
                profit = ((current_df.loc[T1_max_price_index, 'Price']) * eff_1) - ((current_df.loc[T1_min_price_index, 'Price']) / eff_2)
                prices.append((T1_min_price_index, current_df.loc[T1_min_price_index, 'Price'], T1_max_price_index, current_df.loc[T1_max_price_index, 'Price'], profit))
        
        # Repeat similar logic for T2
        current_df_before_min = current_df[current_df.index < T1_min_price_index]
        current_Q_A_before_min = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index < T1_min_price_index)]
        max_price_index_before_min = None
        if not current_Q_A_before_min.empty:
            max_price_index_before_min = current_Q_A_before_min['Price'].idxmax()
        prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index_before_min)]
        min_price_index2 = None
        max_price_index2 = None
        if len(prices_before_max) > 0:
            min_price_index2 = prices_before_max['Price'].idxmin()

        T2_max_price_index = None
        T2_min_price_index = None
        if max_price_index_before_min is not None and min_price_index2 is not None:
            T2_max_price_index = max_price_index_before_min
            T2_min_price_index = min_price_index2
            Exp_profit_T2 = ((current_Q_A.loc[T2_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[T2_min_price_index, 'Price']) / eff_2)
            if Exp_profit_T2 >= 0:
                profit = ((current_df.loc[T2_max_price_index, 'Price']) * eff_1) - ((current_df.loc[T2_min_price_index, 'Price']) / eff_2)
                prices.append((T2_min_price_index, current_df.loc[T2_min_price_index, 'Price'], T2_max_price_index, current_df.loc[T2_max_price_index, 'Price'], profit))
        
        # Repeat similar logic for T3
        current_df_after_T1max = current_df[current_df.index > T1_max_price_index]
        current_Q_A_after_T1max = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > T1_max_price_index)]
        max_price_index_after_T1max = None
        if not current_Q_A_after_T1max.empty:
            max_price_index_after_T1max = current_Q_A_after_T1max['Price'].idxmax()
        T3_prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index_after_T1max) & (Q_B_Preds.index > T1_max_price_index)]
        min_price_index3 = None
        max_price_index3 = None
        if len(T3_prices_before_max) > 0:
            min_price_index3 = T3_prices_before_max['Price'].idxmin()
        
        T3_max_price_index = None
        T3_min_price_index = None
        if max_price_index_after_T1max is not None and min_price_index3 is not None:
            T3_max_price_index = max_price_index_after_T1max
            T3_min_price_index = min_price_index3
            Exp_profit_T3 = ((current_Q_A.loc[T3_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[T3_min_price_index, 'Price']) / eff_2)
            if Exp_profit_T3 >= 0:
                profit = ((current_df.loc[T3_max_price_index, 'Price']) * eff_1) - ((current_df.loc[T3_min_price_index, 'Price']) / eff_2)
                prices.append((T3_min_price_index, current_df.loc[T3_min_price_index, 'Price'], T3_max_price_index, current_df.loc[T3_max_price_index, 'Price'], profit))

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
PF_DAM     =run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r , Q_B_Preds=Y_r,  eff_1=1, eff_2=1)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

TS2 BM

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/knn_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/MH_A_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
# dat1=dat1.iloc[456:,:].reset_index(drop=True)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    day_index = df['level_0'].unique()

    for day in day_index:
        current_df = df[df['level_0'] == day]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == day]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == day]
        
        # Find the maximum price for that day          
        max_price_index = current_Q_A['Price'].idxmax()
        prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index)]
        min_price_index = current_Q_B['Price'].idxmin()
        prices_after_min = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > min_price_index)]

        min_price_index1 = None
        max_price_index1 = None
        
        if len(prices_before_max) > 0:
            min_price_index1 = prices_before_max['Price'].idxmin()
        if len(prices_after_min) > 0:
            max_price_index1 = prices_after_min['Price'].idxmax()

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                T1_max_price_index = max_price_index
                T1_min_price_index = min_price_index1
            else:
                T1_max_price_index = max_price_index1
                T1_min_price_index = min_price_index
        elif max_price_index is not None and min_price_index1 is not None:
            T1_max_price_index = max_price_index
            T1_min_price_index = min_price_index1
        elif max_price_index1 is not None and min_price_index is not None:
            T1_max_price_index = max_price_index1
            T1_min_price_index = min_price_index

        # Calculate expected profit using predictions for T1
        if T1_max_price_index in current_Q_A.index and T1_min_price_index in current_Q_B.index:
            Exp_profit_T1 = ((current_Q_A.loc[T1_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[T1_min_price_index, 'Price']) / eff_2)
            if Exp_profit_T1 >= 0:
                profit = ((current_df.loc[T1_max_price_index, 'Price']) * eff_1) - ((current_df.loc[T1_min_price_index, 'Price']) / eff_2)
                prices.append((T1_min_price_index, current_df.loc[T1_min_price_index, 'Price'], T1_max_price_index, current_df.loc[T1_max_price_index, 'Price'], profit))
        
        # Repeat similar logic for T2
        current_df_before_min = current_df[current_df.index < T1_min_price_index]
        current_Q_A_before_min = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index < T1_min_price_index)]
        max_price_index_before_min = None
        if not current_Q_A_before_min.empty:
            max_price_index_before_min = current_Q_A_before_min['Price'].idxmax()
        prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index_before_min)]
        min_price_index2 = None
        max_price_index2 = None
        if len(prices_before_max) > 0:
            min_price_index2 = prices_before_max['Price'].idxmin()

        T2_max_price_index = None
        T2_min_price_index = None
        if max_price_index_before_min is not None and min_price_index2 is not None:
            T2_max_price_index = max_price_index_before_min
            T2_min_price_index = min_price_index2
            Exp_profit_T2 = ((current_Q_A.loc[T2_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[T2_min_price_index, 'Price']) / eff_2)
            if Exp_profit_T2 >= 0:
                profit = ((current_df.loc[T2_max_price_index, 'Price']) * eff_1) - ((current_df.loc[T2_min_price_index, 'Price']) / eff_2)
                prices.append((T2_min_price_index, current_df.loc[T2_min_price_index, 'Price'], T2_max_price_index, current_df.loc[T2_max_price_index, 'Price'], profit))
        
        # Repeat similar logic for T3
        current_df_after_T1max = current_df[current_df.index > T1_max_price_index]
        current_Q_A_after_T1max = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > T1_max_price_index)]
        max_price_index_after_T1max = None
        if not current_Q_A_after_T1max.empty:
            max_price_index_after_T1max = current_Q_A_after_T1max['Price'].idxmax()
        T3_prices_before_max = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < max_price_index_after_T1max) & (Q_B_Preds.index > T1_max_price_index)]
        min_price_index3 = None
        max_price_index3 = None
        if len(T3_prices_before_max) > 0:
            min_price_index3 = T3_prices_before_max['Price'].idxmin()
        
        T3_max_price_index = None
        T3_min_price_index = None
        if max_price_index_after_T1max is not None and min_price_index3 is not None:
            T3_max_price_index = max_price_index_after_T1max
            T3_min_price_index = min_price_index3
            Exp_profit_T3 = ((current_Q_A.loc[T3_max_price_index, 'Price']) * eff_1) - ((current_Q_B.loc[T3_min_price_index, 'Price']) / eff_2)
            if Exp_profit_T3 >= 0:
                profit = ((current_df.loc[T3_max_price_index, 'Price']) * eff_1) - ((current_df.loc[T3_min_price_index, 'Price']) / eff_2)
                prices.append((T3_min_price_index, current_df.loc[T3_min_price_index, 'Price'], T3_max_price_index, current_df.loc[T3_max_price_index, 'Price'], profit))

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])


r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=1, eff_2=1)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

TS3 DAM

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/knn_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/SH_A_Q_DAM_1-12.csv")
dat1 = pd.DataFrame(dat)
# dat1 = dat1.iloc[152:, :].reset_index(drop=True)


column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

#flexible time stamp order. will limit how much can be charged/discharged at any given time
def process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index):
    if min_price_index < max_price_index:
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
    elif min_price_index > max_price_index:
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))        
    return charge_level

#finding all the possible trade pairs within a identified min/max price index using flexible time stamp order
def recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, day):
    if len(remaining_prices_A) <= 1:
        return charge_level

    max_price_index = remaining_prices_A['Price'].idxmax()
    min_price_index = remaining_prices_B['Price'].idxmin()

    if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
        charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)

    smaller_index = min(min_price_index, max_price_index)
    larger_index = max(min_price_index, max_price_index)

    remaining_prices_A = current_Q_A[(current_Q_A['level_0'] == day) & (current_Q_A.index > smaller_index) & (current_Q_A.index < larger_index)]
    remaining_prices_B = current_Q_B[(current_Q_B['level_0'] == day) & (current_Q_B.index > smaller_index) & (current_Q_B.index < larger_index)]

    charge_level = recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, day)

    return charge_level


# Updated run_electricity_strategy function with trade limit
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2, capacity, charge_level, ramp_rate, min_charge_level, profit_threshold=-10, max_trades_per_day=2):
    prices = []
    day_index = df['level_0'].unique()
    
    for day in day_index:
        # Initialize trade counter for the day
        trades_today = 0
        current_df = df[df['level_0'] == day]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == day]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == day]

        # If the maximum number of trades for the day has been reached, skip further processing
        if trades_today >= max_trades_per_day:
            continue

        # First trading pair (T1) logic
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        # Calculate expected profit for first trade (T1)
        expected_profit_T1 = (current_Q_A.loc[max_price_index, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index, 'Price'] / eff_2)
        if expected_profit_T1 >= profit_threshold:
            charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)
            trades_today += 1

        # Ensure further trades only happen if daily trade limit has not been reached
        if trades_today < max_trades_per_day:
            # Second trade before T1
            prices_before_T1_A = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index < min_price_index)]
            prices_before_T1_B = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < min_price_index)]
            
            if len(prices_before_T1_A) > 1 and len(prices_before_T1_B) > 1:
                max_price_index_2 = prices_before_T1_A['Price'].idxmax()
                min_price_index_2 = prices_before_T1_B['Price'].idxmin()
                
                expected_profit_T2 = (current_Q_A.loc[max_price_index_2, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index_2, 'Price'] / eff_2)
                if expected_profit_T2 >= profit_threshold:
                    charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_2, max_price_index_2)
                    trades_today += 1

        # Stop further trades if daily trade limit is reached
        if trades_today >= max_trades_per_day:
            continue

        # Third trade after T1
        prices_after_T1_A = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > max_price_index)]
        prices_after_T1_B = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index > max_price_index)]
        
        if len(prices_after_T1_A) > 1 and len(prices_after_T1_B) > 1:
            max_price_index_3 = prices_after_T1_A['Price'].idxmax()
            min_price_index_3 = prices_after_T1_B['Price'].idxmin()
            
            expected_profit_T3 = (current_Q_A.loc[max_price_index_3, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index_3, 'Price'] / eff_2)
            if expected_profit_T3 >= profit_threshold:
                charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_3, max_price_index_3)
                trades_today += 1

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])

r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)
PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=-10, max_trades_per_day=2)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

TS3 BM

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/knn_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/MH_A_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
# dat1=dat1.iloc[456:,:].reset_index(drop=True)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]



def process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index):
    if min_price_index < max_price_index:
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))
    elif min_price_index > max_price_index:
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        profit = (current_df.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index, 'Price'],  max_price_index, current_df.loc[max_price_index, 'Price'], profit, charge_level))        
    return charge_level

#finding all the possible trade pairs within a identified min/max price index using flexible time stamp order
def recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, day):
    if len(remaining_prices_A) <= 1:
        return charge_level

    max_price_index = remaining_prices_A['Price'].idxmax()
    min_price_index = remaining_prices_B['Price'].idxmin()

    if current_Q_B.loc[min_price_index, 'Price'] < current_Q_A.loc[max_price_index, 'Price']:
        charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)

    smaller_index = min(min_price_index, max_price_index)
    larger_index = max(min_price_index, max_price_index)

    remaining_prices_A = current_Q_A[(current_Q_A['level_0'] == day) & (current_Q_A.index > smaller_index) & (current_Q_A.index < larger_index)]
    remaining_prices_B = current_Q_B[(current_Q_B['level_0'] == day) & (current_Q_B.index > smaller_index) & (current_Q_B.index < larger_index)]

    charge_level = recursive_process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, current_Q_A, current_Q_B, day)

    return charge_level


# Updated run_electricity_strategy function with trade limit
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2, capacity, charge_level, ramp_rate, min_charge_level, profit_threshold=0, max_trades_per_day=2):
    prices = []
    day_index = df['level_0'].unique()
    
    for day in day_index:
        # Initialize trade counter for the day
        trades_today = 0
        current_df = df[df['level_0'] == day]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == day]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == day]

        # If the maximum number of trades for the day has been reached, skip further processing
        if trades_today >= max_trades_per_day:
            continue

        # First trading pair (T1) logic
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()

        # Calculate expected profit for first trade (T1)
        expected_profit_T1 = (current_Q_A.loc[max_price_index, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index, 'Price'] / eff_2)
        if expected_profit_T1 >= profit_threshold:
            charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)
            trades_today += 1

        # Ensure further trades only happen if daily trade limit has not been reached
        if trades_today < max_trades_per_day:
            # Second trade before T1
            prices_before_T1_A = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index < min_price_index)]
            prices_before_T1_B = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index < min_price_index)]
            
            if len(prices_before_T1_A) > 1 and len(prices_before_T1_B) > 1:
                max_price_index_2 = prices_before_T1_A['Price'].idxmax()
                min_price_index_2 = prices_before_T1_B['Price'].idxmin()
                
                expected_profit_T2 = (current_Q_A.loc[max_price_index_2, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index_2, 'Price'] / eff_2)
                if expected_profit_T2 >= profit_threshold:
                    charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_2, max_price_index_2)
                    trades_today += 1

        # Stop further trades if daily trade limit is reached
        if trades_today >= max_trades_per_day:
            continue

        # Third trade after T1
        prices_after_T1_A = Q_A_Preds[(Q_A_Preds['level_0'] == day) & (Q_A_Preds.index > max_price_index)]
        prices_after_T1_B = Q_B_Preds[(Q_B_Preds['level_0'] == day) & (Q_B_Preds.index > max_price_index)]
        
        if len(prices_after_T1_A) > 1 and len(prices_after_T1_B) > 1:
            max_price_index_3 = prices_after_T1_A['Price'].idxmax()
            min_price_index_3 = prices_after_T1_B['Price'].idxmin()
            
            expected_profit_T3 = (current_Q_A.loc[max_price_index_3, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index_3, 'Price'] / eff_2)
            if expected_profit_T3 >= profit_threshold:
                charge_level = process_prices(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index_3, max_price_index_3)
                trades_today += 1

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)
PF_BM     =run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r,  Q_B_Preds=Y_r,  eff_1=0.8, eff_2=0.98, capacity=1,charge_level=0, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=1)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

TS3 Dual

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

num_rows = 365 * 48
df = pd.DataFrame(index=range(num_rows))

# Add the 'level_0' column
df['level_0'] = np.repeat(np.arange(366), 48)[:num_rows]


date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/knn_Q_1-12.csv")
dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/rf_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Conformal_Prediction/BM/lgbm_Q_1-12.csv")
# dat1 = pd.read_csv("/home/ciaran/Desktop/Model_preds/MH_A_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
# dat1=dat1.iloc[456:,:].reset_index(drop=True)


dat1=pd.concat([dat1, df],axis=1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90_BM=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r_BM=dat1[column_names].dropna().stack().reset_index()
Y_r_BM["Price"]=Y_r_BM.iloc[:,2:3]
Q_10_BM["Price"]=Q_10_BM.iloc[:,2:3]
Q_30_BM["Price"]=Q_30_BM.iloc[:,2:3]
Q_50_BM["Price"]=Q_50_BM.iloc[:,2:3]
Q_70_BM["Price"]=Q_70_BM.iloc[:,2:3]
Q_90_BM["Price"]=Q_90_BM.iloc[:,2:3]

Y_r_BM  = Y_r_BM.iloc[:,1:]
Q_10_BM = Q_10_BM.iloc[:,1:]
Q_30_BM = Q_30_BM.iloc[:,1:]
Q_50_BM = Q_50_BM.iloc[:,1:]
Q_70_BM = Q_70_BM.iloc[:,1:]
Q_90_BM = Q_90_BM.iloc[:,1:]

Y_r_BM= pd.concat([Y_r_BM, df],axis=1)
Q_10_BM= pd.concat([Q_10_BM, df],axis=1)
Q_30_BM= pd.concat([Q_30_BM, df],axis=1)
Q_50_BM= pd.concat([Q_50_BM, df],axis=1)
Q_70_BM= pd.concat([Q_70_BM, df],axis=1)
Q_90_BM= pd.concat([Q_90_BM, df],axis=1)


date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/knn_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Conformal_Prediction/DAM/lgbm_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/SH_A_Q_DAM_1-12.csv")
dat1 = pd.DataFrame(dat)
# dat1 = dat1.iloc[152:, :].reset_index(drop=True)


start = 0
end = 17519
step = 2

values = list(range(start, end+1, step))
df1 = pd.DataFrame({'P_dam': values})

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90_DAM=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r_DAM=dat1[column_names].dropna().stack().reset_index()
Y_r_DAM=Y_r_DAM.iloc[:,:]

Y_r_DAM["Price"]=Y_r_DAM.iloc[:,2:3]
Q_10_DAM["Price"]=Q_10_DAM.iloc[:,2:3]
Q_30_DAM["Price"]=Q_30_DAM.iloc[:,2:3]
Q_50_DAM["Price"]=Q_50_DAM.iloc[:,2:3]
Q_70_DAM["Price"]=Q_70_DAM.iloc[:,2:3]
Q_90_DAM["Price"]=Q_90_DAM.iloc[:,2:3]

Y_r_DAM= pd.concat([Y_r_DAM, df1], axis=1)
Q_10_DAM= pd.concat([Q_10_DAM, df1], axis=1)
Q_30_DAM= pd.concat([Q_30_DAM, df1], axis=1)
Q_50_DAM= pd.concat([Q_50_DAM, df1], axis=1)
Q_70_DAM= pd.concat([Q_70_DAM, df1], axis=1)
Q_90_DAM= pd.concat([Q_90_DAM, df1], axis=1)



def process_prices_DAM(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index):
    if min_price_index < max_price_index:
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        profit = (current_df.loc[max_price_index // 2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index // 2, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index // 2, 'Price'], max_price_index, current_df.loc[max_price_index // 2, 'Price'], profit, charge_level))
    elif min_price_index > max_price_index:
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        profit = (current_df.loc[max_price_index // 2, 'Price'] * bottleneck_2 * eff_1) - ((current_df.loc[min_price_index // 2, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df.loc[min_price_index // 2, 'Price'], max_price_index, current_df.loc[max_price_index // 2, 'Price'], profit, charge_level))
    return charge_level


def process_prices_BM(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df_bm, min_price_index, max_price_index):
    if min_price_index < max_price_index:
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        profit = (current_df_bm.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df_bm.loc[min_price_index, 'Price'], max_price_index, current_df_bm.loc[max_price_index, 'Price'], profit, charge_level))
    elif min_price_index > max_price_index:
        bottleneck_2 = min(charge_level - min_charge_level, ramp_rate)
        charge_level -= bottleneck_2
        bottleneck_1 = min(capacity - charge_level, ramp_rate)
        charge_level += bottleneck_1
        profit = (current_df_bm.loc[max_price_index, 'Price'] * bottleneck_2 * eff_1) - ((current_df_bm.loc[min_price_index, 'Price'] * bottleneck_1) / eff_2)
        prices.append((min_price_index, current_df_bm.loc[min_price_index, 'Price'], max_price_index, current_df_bm.loc[max_price_index, 'Price'], profit, charge_level))
    return charge_level

def process_recursive_DAM(remaining_prices, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, level_0, current_Q_A, current_Q_B, Q_A_Preds, Q_B_Preds):
    if len(remaining_prices) > 1:
        max_price_index = remaining_prices['Price'].idxmax()
        min_price_index = remaining_prices['Price'].idxmin()
        max_price_index = remaining_prices.loc[max_price_index, 'P_dam']
        min_price_index = remaining_prices.loc[min_price_index, 'P_dam']
        
        if current_Q_B.loc[min_price_index / 2, 'Price'] < current_Q_A.loc[max_price_index / 2, 'Price']:
            charge_level = process_prices_DAM(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)
        else:
            return charge_level  

        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        remaining_prices = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        remaining_prices_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        remaining_prices_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]

        charge_level = process_recursive_DAM(remaining_prices, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, remaining_prices_A, remaining_prices_B, level_0, current_Q_A, current_Q_B, Q_A_Preds, Q_B_Preds)
    
    return charge_level

def process_recursive_bm(remaining_prices, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df_bm, remaining_prices_A, remaining_prices_B, level_0, current_Q_A_bm, current_Q_B_bm, Q_A_Preds_bm, Q_B_Preds_bm):
    if len(remaining_prices) <= 1:
        return charge_level

    max_price_index = remaining_prices['Price'].idxmax()
    min_price_index = remaining_prices['Price'].idxmin()

    if current_Q_B_bm.loc[min_price_index, 'Price'] < current_Q_A_bm.loc[max_price_index, 'Price']:
        charge_level = process_prices_BM(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df_bm, min_price_index, max_price_index)
    else:
        return charge_level

    smaller_index = min(min_price_index, max_price_index)
    larger_index = max(min_price_index, max_price_index)
    remaining_prices_A = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index < larger_index)]
    remaining_prices_B = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index < larger_index)]

    charge_level = process_recursive_bm(remaining_prices_A, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df_bm, remaining_prices_A, remaining_prices_B, level_0, current_Q_A_bm, current_Q_B_bm, Q_A_Preds_bm, Q_B_Preds_bm)
    
    return charge_level


def dual_strat(df, df_bm, Q_A_Preds, Q_B_Preds, Q_A_Preds_bm, Q_B_Preds_bm, eff_1, eff_2, capacity, charge_level, ramp_rate, min_charge_level, profit_threshold=10, max_trades_per_day=2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        trades_today = 0  # Reset daily trade counter
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        current_df_bm = df_bm[df_bm['level_0'] == level_0]
        current_Q_A_bm = Q_A_Preds_bm[Q_A_Preds_bm['level_0'] == level_0]
        current_Q_B_bm = Q_B_Preds_bm[Q_B_Preds_bm['level_0'] == level_0]  
        
        # DAM Trade Initialization
        max_price_index = current_Q_A['Price'].idxmax()
        min_price_index = current_Q_B['Price'].idxmin()
        max_price_index = current_Q_A.loc[max_price_index, 'P_dam']
        min_price_index = current_Q_B.loc[min_price_index, 'P_dam']

        # Calculate Expected Profit for DAM Initial Trade
        expected_profit_DAM = (current_Q_A.loc[max_price_index / 2, 'Price'] * eff_1) - (current_Q_B.loc[min_price_index / 2, 'Price'] / eff_2)
        
        if expected_profit_DAM >= profit_threshold:
            charge_level = process_prices_DAM(charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, min_price_index, max_price_index)
            trades_today += 1
        else:
            continue

        # Ensure trade limit has not been reached
        if trades_today >= max_trades_per_day:
            continue
        
        # Recursive DAM Optimization
        smaller_index = min(min_price_index, max_price_index)
        larger_index = max(min_price_index, max_price_index)
        DAM_Intraday = current_df[(current_df['P_dam'] > smaller_index) & (current_df['P_dam'] < larger_index)]
        DAM_Intraday_Q1 = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds['P_dam'] > smaller_index) & (Q_A_Preds['P_dam'] < larger_index)]
        DAM_Intraday_Q2 = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds['P_dam'] > smaller_index) & (Q_B_Preds['P_dam'] < larger_index)]

        # Recursive DAM Optimization with Profit Threshold
        if len(DAM_Intraday) > 1:
            max_price_index_DAM = DAM_Intraday_Q1['Price'].idxmax()
            min_price_index_DAM = DAM_Intraday_Q2['Price'].idxmin()
            expected_profit_DAM_Intraday = (DAM_Intraday_Q1.loc[max_price_index_DAM, 'Price'] * eff_1) - (DAM_Intraday_Q2.loc[min_price_index_DAM, 'Price'] / eff_2)

            if expected_profit_DAM_Intraday >= profit_threshold:
                charge_level = process_recursive_DAM(DAM_Intraday, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df, DAM_Intraday_Q1, DAM_Intraday_Q2, level_0, current_Q_A, current_Q_B, Q_A_Preds, Q_B_Preds)

        # BM Trade Before DAM with Profit Threshold
        if trades_today < max_trades_per_day:
            BM_before_DAM = current_df_bm[(current_df_bm.index < smaller_index) & (current_df_bm.index < larger_index)]
            BM_before_DAM_Q1 = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index < smaller_index) & (Q_A_Preds_bm.index < larger_index)]
            BM_before_DAM_Q2 = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index < smaller_index) & (Q_B_Preds_bm.index < larger_index)]

            if len(BM_before_DAM) > 1:
                max_price_index_BM_before = BM_before_DAM_Q1['Price'].idxmax()
                min_price_index_BM_before = BM_before_DAM_Q2['Price'].idxmin()
                expected_profit_BM_before = (BM_before_DAM_Q1.loc[max_price_index_BM_before, 'Price'] * eff_1) - (BM_before_DAM_Q2.loc[min_price_index_BM_before, 'Price'] / eff_2)

                if expected_profit_BM_before >= profit_threshold:
                    charge_level = process_recursive_bm(BM_before_DAM, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df_bm, BM_before_DAM_Q1, BM_before_DAM_Q2, level_0, current_Q_A_bm, current_Q_B_bm, Q_A_Preds_bm, Q_B_Preds_bm)
                    trades_today += 1

        # BM Trade After DAM with Profit Threshold
        if trades_today < max_trades_per_day:
            BM_after_DAM = current_df_bm[(current_df_bm.index > smaller_index) & (current_df_bm.index > larger_index)]
            BM_after_DAM_Q1 = Q_A_Preds_bm[(Q_A_Preds_bm['level_0'] == level_0) & (Q_A_Preds_bm.index > smaller_index) & (Q_A_Preds_bm.index > larger_index)]
            BM_after_DAM_Q2 = Q_B_Preds_bm[(Q_B_Preds_bm['level_0'] == level_0) & (Q_B_Preds_bm.index > smaller_index) & (Q_B_Preds_bm.index > larger_index)]

            if len(BM_after_DAM) > 1:
                max_price_index_BM_after = BM_after_DAM_Q1['Price'].idxmax()
                min_price_index_BM_after = BM_after_DAM_Q2['Price'].idxmin()
                expected_profit_BM_after = (BM_after_DAM_Q1.loc[max_price_index_BM_after, 'Price'] * eff_1) - (BM_after_DAM_Q2.loc[min_price_index_BM_after, 'Price'] / eff_2)

                if expected_profit_BM_after >= profit_threshold:
                    charge_level = process_recursive_bm(BM_after_DAM, charge_level, capacity, ramp_rate, min_charge_level, eff_1, eff_2, prices, current_df_bm, BM_after_DAM_Q1, BM_after_DAM_Q2, level_0, current_Q_A_bm, current_Q_B_bm, Q_A_Preds_bm, Q_B_Preds_bm)
                    trades_today += 1


    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit', 'charge Level'])


r_dam_bm_50_50=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
r_dam_bm_10_30=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_30_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_30_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
r_dam_bm_30_50=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_50_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_50_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
r_dam_bm_50_70=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_50_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_50_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
r_dam_bm_70_90=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_70_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_70_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
r_dam_bm_30_70=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_30_DAM, Q_B_Preds=Q_70_DAM, Q_A_Preds_bm=Q_30_BM, Q_B_Preds_bm=Q_70_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
r_dam_bm_10_90=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Q_10_DAM, Q_B_Preds=Q_90_DAM, Q_A_Preds_bm=Q_10_BM, Q_B_Preds_bm=Q_90_BM, eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)
PF_dam_bm=dual_strat(df=Y_r_DAM, df_bm=Y_r_BM, Q_A_Preds=Y_r_DAM, Q_B_Preds=Y_r_DAM, Q_A_Preds_bm=Y_r_BM, Q_B_Preds_bm=Y_r_BM,          eff_1=0.8, eff_2=0.98, capacity=1,charge_level=1, ramp_rate=1, min_charge_level=0, profit_threshold=0, max_trades_per_day=2)

print(np.round(sum(r_dam_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_dam_bm.iloc[:,4:5].values),2))